In [1]:
import gc

from pprint import pprint

import pandas as pd
import numpy as np

from sklearn.decomposition import TruncatedSVD

from cv import run_cv_model
from utils import print_step, rmse
from cache import get_data, is_in_cache, load_cache, save_in_cache

from keras.layers import Dense, Input, Embedding, Dropout, Activation, concatenate, BatchNormalization, Flatten, Concatenate, Conv1D
from keras.models import Model
from keras import backend as K
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

from sklearn.model_selection import KFold

/home/sijunhe/Environment/ML_py_36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
print('~~~~~~~~~~~~~~~~~~~~~~~~')
print_step('Importing Data 1/15')
train, test = get_data()

print('~~~~~~~~~~~~~~~')
print_step('Subsetting')
target = train['deal_probability']
train_id = train['item_id']
test_id = test['item_id']
train.drop(['deal_probability', 'item_id'], axis=1, inplace=True)
test.drop(['item_id'], axis=1, inplace=True)

print('~~~~~~~~~~~~~~~~~~~~~~~~')
print_step('Importing Data 2/15')
train_fe, test_fe = load_cache('data_with_fe')

print_step('Importing Data 2/15')
train_ridge, test_ridge = load_cache('tfidf_ridges')
drops = [c for c in train_ridge.columns if 'svd' in c or 'tfidf' in c]
train_ridge.drop(drops, axis=1, inplace=True)
test_ridge.drop(drops, axis=1, inplace=True)
train_ = train_ridge
test_ = test_ridge

print_step('Importing Data 2/15')
train_['parent_category_name'] = train_fe['parent_category_name']
test_['parent_category_name'] = test_fe['parent_category_name']
train_['price'] = train_fe['price']
test_['price'] = test_fe['price']

# print_step('Importing Data 3/15 1/3')
# train_base_lgb, test_base_lgb = load_cache('base_lgb')
# print_step('Importing Data 3/15 2/3')
# train_['base_lgb'] = train_base_lgb['base_lgb']
# print_step('Importing Data 3/15 3/3')
# test_['base_lgb'] = test_base_lgb['base_lgb']

print_step('Importing Data 3/15 1/3')
train_base_lgb, test_base_lgb = load_cache('base_lgb2')
print_step('Importing Data 3/15 2/3')
train_['base_lgb2'] = train_base_lgb['base_lgb2']
print_step('Importing Data 3/15 3/3')
test_['base_lgb2'] = test_base_lgb['base_lgb2']

# train_te_lgb, test_te_lgb = load_cache('te_lgb')
# print_step('Importing Data 3/15 2/3')
# train_['te_lgb'] = train_te_lgb['te_lgb']
# print_step('Importing Data 3/15 3/3')
# test_['te_lgb'] = test_te_lgb['te_lgb']

# print_step('Importing Data 3/15 1/3')
# train_te_lgb2, test_te_lgb2 = load_cache('te_lgb2')
# print_step('Importing Data 3/15 2/3')
# train_['te_lgb2'] = train_te_lgb2['te_lgb']
# print_step('Importing Data 3/15 3/3')
# test_['te_lgb2'] = test_te_lgb2['te_lgb']

print_step('Importing Data 3/15 1/3')
train_te_lgb2, test_te_lgb2 = load_cache('te_lgb3')
print_step('Importing Data 3/15 2/3')
train_['te_lgb3'] = train_te_lgb2['te_lgb3']
print_step('Importing Data 3/15 3/3')
test_['te_lgb3'] = test_te_lgb2['te_lgb3']

# print_step('Importing Data 3/15 1/3')
# train_te_lgb2, test_te_lgb2 = load_cache('te_lgb_poisson')
# print_step('Importing Data 3/15 2/3')
# train_['te_lgb_poisson'] = train_te_lgb2['te_lgb_poisson']
# print_step('Importing Data 3/15 3/3')
# test_['te_lgb_poisson'] = test_te_lgb2['te_lgb_poisson']

print_step('Importing Data 3/15 1/3')
train_ryan_lgbm_v29, test_ryan_lgbm_v29 = load_cache('ryan_lgbm_v29')
print_step('Importing Data 3/15 2/3')
train_['ryan_lgbm_v29'] = train_ryan_lgbm_v29['oof_lgbm']
print_step('Importing Data 3/15 3/3')
test_['ryan_lgbm_v29'] = test_ryan_lgbm_v29['oof_lgbm']

print_step('Importing Data 3/15 1/3')
train_ryan_lgbm_v29, test_ryan_lgbm_v29 = load_cache('ryan_lgbm_v33')
print_step('Importing Data 3/15 2/3')
train_['ryan_lgbm_v33'] = train_ryan_lgbm_v29['oof_lgbm']
print_step('Importing Data 3/15 3/3')
test_['ryan_lgbm_v33'] = test_ryan_lgbm_v29['oof_lgbm']

print_step('Importing Data 3/15 1/3')
train_ryan_lgbm_v29, test_ryan_lgbm_v29 = load_cache('ryan_lgbm_v36')
print_step('Importing Data 3/15 2/3')
train_['ryan_lgbm_v36'] = train_ryan_lgbm_v29['oof_lgbm']
print_step('Importing Data 3/15 3/3')
test_['ryan_lgbm_v36'] = test_ryan_lgbm_v29['oof_lgbm']

# print_step('Importing Data 3/15 1/3')
# train_ridge_lgb, test_ridge_lgb = load_cache('ridge_lgb')
# print_step('Importing Data 3/15 2/3')
# train_['ridge_lgb'] = train_ridge_lgb['ridge_lgb']
# print_step('Importing Data 3/15 3/3')
# test_['ridge_lgb'] = test_ridge_lgb['ridge_lgb']

# print_step('Importing Data 3/15 1/3')
# train_ridge_lgb, test_ridge_lgb = load_cache('ridge_lgb2')
# print_step('Importing Data 3/15 2/3')
# train_['ridge_lgb2'] = train_ridge_lgb['ridge_lgb2']
# print_step('Importing Data 3/15 3/3')
# test_['ridge_lgb2'] = test_ridge_lgb['ridge_lgb2']

print_step('Importing Data 3/15 1/3')
train_ridge_lgb, test_ridge_lgb = load_cache('ridge_lgb3')
print_step('Importing Data 3/15 2/3')
train_['ridge_lgb3'] = train_ridge_lgb['ridge_lgb3']
print_step('Importing Data 3/15 3/3')
test_['ridge_lgb3'] = test_ridge_lgb['ridge_lgb3']

print_step('Importing Data 3/15 1/3')
train_ridge_lgb, test_ridge_lgb = load_cache('ridge_lgb_poisson')
print_step('Importing Data 3/15 2/3')
train_['ridge_lgb_poisson'] = train_ridge_lgb['ridge_lgb_poisson']
print_step('Importing Data 3/15 3/3')
test_['ridge_lgb_poisson'] = test_ridge_lgb['ridge_lgb_poisson']

print_step('Importing Data 4/15 1/4')
train_pcat_ridge, test_pcat_ridge = load_cache('parent_cat_ridges')
print_step('Importing Data 4/15 2/4')
train_pcat_ridge = train_pcat_ridge[[c for c in train_pcat_ridge.columns if 'ridge' in c]]
test_pcat_ridge = test_pcat_ridge[[c for c in test_pcat_ridge.columns if 'ridge' in c]]
print_step('Importing Data 4/15 3/4')
train_ = pd.concat([train_, train_pcat_ridge], axis=1)
print_step('Importing Data 4/15 4/4')
test_ = pd.concat([test_, test_pcat_ridge], axis=1)

print_step('Importing Data 5/15 1/4')
train_rcat_ridge, test_rcat_ridge = load_cache('parent_regioncat_ridges')
print_step('Importing Data 5/15 2/4')
train_rcat_ridge = train_rcat_ridge[[c for c in train_rcat_ridge.columns if 'ridge' in c]]
test_rcat_ridge = test_rcat_ridge[[c for c in test_rcat_ridge.columns if 'ridge' in c]]
print_step('Importing Data 5/15 3/4')
train_ = pd.concat([train_, train_rcat_ridge], axis=1)
print_step('Importing Data 5/15 4/4')
test_ = pd.concat([test_, test_rcat_ridge], axis=1)

print_step('Importing Data 6/15 1/4')
train_catb_ridge, test_catb_ridge = load_cache('cat_bin_ridges')
print_step('Importing Data 6/15 2/4')
train_catb_ridge = train_catb_ridge[[c for c in train_catb_ridge.columns if 'ridge' in c]]
test_catb_ridge = test_catb_ridge[[c for c in test_catb_ridge.columns if 'ridge' in c]]
print_step('Importing Data 6/15 3/4')
train_ = pd.concat([train_, train_catb_ridge], axis=1)
print_step('Importing Data 6/15 4/4')
test_ = pd.concat([test_, test_catb_ridge], axis=1)

print_step('Importing Data 7/15 1/3')
train_deep_lgb, test_deep_lgb = load_cache('deep_lgb')
print_step('Importing Data 7/15 2/3')
train_['deep_lgb'] = train_deep_lgb['deep_lgb']
print_step('Importing Data 7/15 3/3')
test_['deep_lgb'] = test_deep_lgb['deep_lgb']

print_step('Importing Data 7/15 1/3')
train_deep_lgb, test_deep_lgb = load_cache('deep_lgb2')
print_step('Importing Data 7/15 2/3')
train_['deep_lgb2'] = train_deep_lgb['deep_lgb2']
print_step('Importing Data 7/15 3/3')
test_['deep_lgb2'] = test_deep_lgb['deep_lgb2']

print_step('Importing Data 7/15 1/3')
train_deep_lgb, test_deep_lgb = load_cache('deep_lgb3')
print_step('Importing Data 7/15 2/3')
train_['deep_lgb3'] = train_deep_lgb['deep_lgb3']
print_step('Importing Data 7/15 3/3')
test_['deep_lgb3'] = test_deep_lgb['deep_lgb3']

print_step('Importing Data 8/15 1/3')
train_full_text_ridge, test_full_text_ridge = load_cache('full_text_ridge')
print_step('Importing Data 8/15 2/3')
train_['full_text_ridge'] = train_full_text_ridge['full_text_ridge']
print_step('Importing Data 8/15 3/3')
test_['full_text_ridge'] = test_full_text_ridge['full_text_ridge']

print_step('Importing Data 9/15 1/3')
train_complete_ridge, test_complete_ridge = load_cache('complete_ridge')
print_step('Importing Data 9/15 2/3')
train_['complete_ridge'] = train_complete_ridge['complete_ridge']
print_step('Importing Data 9/15 3/3')
test_['complete_ridge'] = test_complete_ridge['complete_ridge']

print_step('Importing Data 9/15 1/3')
train_ryan_ridge, test_ryan_ridge = load_cache('ryan_ridge_sgd_v2')
print_step('Importing Data 9/15 2/3')
train_['ryan_ridge'] = train_ryan_ridge['oof_ridge']
print_step('Importing Data 9/15 3/3')
test_['ryan_ridge'] = test_ryan_ridge['oof_ridge']
print_step('Importing Data 9/15 2/3')
train_['ryan_sgd'] = train_ryan_ridge['oof_sgd']
print_step('Importing Data 9/15 3/3')
test_['ryan_sgd'] = test_ryan_ridge['oof_sgd']

print_step('Importing Data 10/15 1/3')
train_complete_fm, test_complete_fm = load_cache('complete_fm')
print_step('Importing Data 10/15 2/3')
train_['complete_fm'] = train_complete_fm['complete_fm']
print_step('Importing Data 10/15 3/3')
test_['complete_fm'] = test_complete_fm['complete_fm']

print_step('Importing Data 11/15 1/3')
train_tffm2, test_tffm2 = load_cache('tffm2')
print_step('Importing Data 11/15 2/3')
train_['tffm2'] = train_tffm2['tffm2']
print_step('Importing Data 11/15 3/3')
test_['tffm2'] = test_tffm2['tffm2']

print_step('Importing Data 12/15 1/3')
train_tffm3, test_tffm3 = load_cache('tffm3')
print_step('Importing Data 12/15 2/3')
train_['tffm3'] = train_tffm3['tffm3']
print_step('Importing Data 12/15 3/3')
test_['tffm3'] = test_tffm3['tffm3']

print_step('Importing Data 13/15 1/3')
train_cnn_ft, test_cnn_ft = load_cache('CNN_FastText')
print_step('Importing Data 13/15 2/3')
train_['cnn_ft'] = train_cnn_ft['CNN_FastText']
print_step('Importing Data 13/15 3/3')
test_['cnn_ft'] = test_cnn_ft['CNN_FastText']

print_step('Importing Data 14/15 1/3')
train_cnn_ft, test_cnn_ft = load_cache('CNN_FastText_4')
print_step('Importing Data 14/15 2/3')
train_['cnn_ft4'] = train_cnn_ft['CNN_FastText_4']
print_step('Importing Data 14/15 3/3')
test_['cnn_ft4'] = test_cnn_ft['CNN_FastText_4']

print_step('Importing Data 14/15 1/3')
train_cnn_ft, test_cnn_ft = load_cache('RNN_AttentionPooling')
print_step('Importing Data 14/15 2/3')
train_['rnn_at'] = train_cnn_ft['RNN_AttentionPooling']
print_step('Importing Data 14/15 3/3')
test_['rnn_at'] = test_cnn_ft['RNN_AttentionPooling']

print_step('Importing Data 14/15 1/3')
train_cnn_ft, test_cnn_ft = load_cache('RNN_AttentionPooling_img2')
print_step('Importing Data 14/15 2/3')
train_['rnn_at2'] = train_cnn_ft['RNN_AttentionPooling_img2']
print_step('Importing Data 14/15 3/3')
test_['rnn_at2'] = test_cnn_ft['RNN_AttentionPooling_img2']

print_step('Importing Data 14/15 1/3')
train_cnn_ft = pd.read_csv('cache/matt_nn_oof.csv')
test_cnn_ft = pd.read_csv('cache/matt_nn_test.csv')
print_step('Importing Data 14/15 2/3')
train_['matt_nn'] = train_cnn_ft['matt_nn']
print_step('Importing Data 14/15 3/3')
test_['matt_nn'] = test_cnn_ft['deal_probability']

print_step('Importing Data 14/15 1/3')
train_multi = pd.read_csv('cache/matt_multi_nn_oof.csv')
test_multi = pd.read_csv('cache/matt_multi_nn_test.csv')
print_step('Importing Data 14/15 2/3')
train_ = pd.concat([train_, train_multi], axis=1)
train_.drop('item_id', axis=1, inplace=True)
print_step('Importing Data 14/15 3/3')
test_ = pd.concat([test_, test_multi], axis=1)
test_.drop('item_id', axis=1, inplace=True)

# print_step('Importing Data 14/15 1/3')
# train_cnn_ft, test_cnn_ft = load_cache('CNN_binary')
# print_step('Importing Data 14/15 2/3')
# train_['CNN_binary'] = train_cnn_ft['CNN_binary']
# print_step('Importing Data 14/15 3/3')
# test_['CNN_binary'] = test_cnn_ft['CNN_binary']

print_step('Importing Data 14/15 1/3')
train_cnn_ft, test_cnn_ft = load_cache('CNN_binary_PL')
print_step('Importing Data 14/15 2/3')
train_['CNN_binary_PL'] = train_cnn_ft['CNN_binary_PL']
print_step('Importing Data 14/15 3/3')
test_['CNN_binary_PL'] = test_cnn_ft['CNN_binary_PL']

print_step('Importing Data 14/15 1/3')
train_liu_nn, test_liu_nn = load_cache('liu_nn')
print_step('Importing Data 14/15 2/3')
train_['liu_nn'] = train_liu_nn['liu_nn']
print_step('Importing Data 14/15 3/3')
test_['liu_nn'] = test_liu_nn['liu_nn']

print_step('Importing Data 14/15 1/3')
train_liu_nn, test_liu_nn = load_cache('liu_nn2')
print_step('Importing Data 14/15 2/3')
train_['liu_nn2'] = train_liu_nn['liu_nn2']
print_step('Importing Data 14/15 3/3')
test_['liu_nn2'] = test_liu_nn['liu_nn2']

print_step('Importing Data 14/15 1/3')
train_liu_lgb, test_liu_lgb = load_cache('liu_lgb')
print_step('Importing Data 14/15 2/3')
train_['liu_lgb'] = train_liu_lgb['liu_lgb']
print_step('Importing Data 14/15 3/3')
test_['liu_lgb'] = test_liu_lgb['liu_lgb']

~~~~~~~~~~~~~~~~~~~~~~~~
[2018-06-26 22:19:57.361160] Importing Data 1/15
Train shape: (1503424, 18)
Test shape: (508438, 17)
~~~~~~~~~~~~~~~
[2018-06-26 22:20:08.658135] Subsetting
~~~~~~~~~~~~~~~~~~~~~~~~
[2018-06-26 22:20:09.023897] Importing Data 2/15
Test shape: (508438, 61)
Train shape: (1503424, 61)
[2018-06-26 22:20:19.866264] Skipped... Loaded cache/train_data_with_fe.csv and cache/test_data_with_fe.csv from cache!
[2018-06-26 22:20:19.891184] Importing Data 2/15
Test shape: (508438, 63)
Train shape: (1503424, 63)
[2018-06-26 22:20:40.138892] Skipped... Loaded cache/train_tfidf_ridges.csv and cache/test_tfidf_ridges.csv from cache!
[2018-06-26 22:20:40.164816] Importing Data 2/15
[2018-06-26 22:20:40.170143] Importing Data 3/15 1/3
Test shape: (508438, 1)
Train shape: (1503424, 1)
[2018-06-26 22:20:40.456519] Skipped... Loaded cache/train_base_lgb2.csv and cache/test_base_lgb2.csv from cache!
[2018-06-26 22:20:40.456656] Importing Data 3/15 2/3
[2018-06-26 22:20:40.457882] Imp

In [13]:
df = pd.concat([train_,test_],axis=0)
num_train = len(train)
df['price'] = np.log1p(df['price'])
num_cols = [i for i in df.columns if i != 'parent_category_name']
for c in num_cols:
    df[c] = (df[c] - np.mean(df[c]))/np.std(df[c])

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
lbl = LabelEncoder()
cat_cols = ['parent_category_name']
for c in cat_cols:
    df[c] = lbl.fit_transform(df[c])

In [16]:
train_ = df[:num_train]
test_ = df[num_train:]

In [17]:
max_category = max(df['parent_category_name']) + 1
num_cols = [i for i in df.columns if i != 'parent_category_name']

In [22]:
def MLP(): 
        
    K.clear_session()

    parent_cat = Input(shape=[1], name='parent_category_name')
    nums = [Input(shape=[1], name=name) for name in num_cols]
    embeded = Flatten() (Embedding(max_category, 50) (parent_cat))
    x = concatenate(nums + [embeded])
    for num_dense in [256, 64, 16]:
        x = Dense(num_dense, activation='relu') (x)
#         x = Dropout(0.1)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[parent_cat]+nums, outputs=output)
    model.compile(loss = 'mse',
                  metrics=['mse'],
                  optimizer='nadam')
    return model

In [23]:
def get_keras_data(df):
    X = {}
    for c in df.columns:
        X[c] = df[c].values
    return X
def run_cv_model(train, test, target, model_fn, eval_fn):
    kf = KFold(n_splits=5, shuffle=True, random_state=2017)
    fold_splits = kf.split(train)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros(train.shape[0])
    i = 1
    test_X = get_keras_data(test)
    for dev_index, val_index in fold_splits:
        print_step('Started ' + ' fold ' + str(i) + '/5')
        dev_X = get_keras_data(train.loc[dev_index])
        val_X = get_keras_data(train.loc[val_index])
        dev_y, val_y = target[dev_index], target[val_index]
        pred_val_y, pred_test_y = model_fn(dev_X, dev_y, val_X, val_y, test_X)
        pred_val_y = pred_val_y.flatten()
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        cv_score = eval_fn(val_y, pred_val_y)
        cv_scores.append(eval_fn(val_y, pred_val_y))
        print_step(' cv score ' + str(i) + ' : ' + str(cv_score))
        i += 1
    print_step(' cv scores : ' + str(cv_scores))
    print_step(' mean cv score : ' + str(np.mean(cv_scores)))
    print_step(' std cv score : ' + str(np.std(cv_scores)))
    pred_full_test = pred_full_test / 5.0
    results = {'train': pred_train, 'test': pred_full_test,
                'cv': cv_scores}
    return results

In [24]:
def runMLP(train_X, train_y, val_X, val_y, test_X):
    model = MLP()
    
    ### Callbacks
    def schedule(ind):
        return(1e-3/(1.5**(ind)))

    lr = LearningRateScheduler(schedule)
    
#     tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
#                           write_graph=True, write_images=False)
    
    model_checkpoint = ModelCheckpoint(filepath = 'tmp_bst_mdl.hdf5', monitor='val_mean_squared_error', verbose=0, save_best_only = True, mode='min')

#     tb = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
    model.fit(train_X, train_y, 
              validation_data=(val_X, val_y),
              batch_size=256, epochs=4, verbose=2,
             callbacks = [lr, model_checkpoint])
    print_step("Loading best model")
    model.load_weights(filepath = 'tmp_bst_mdl.hdf5')
    print_step('Predict Val 1/2')
    pred_val_y = model.predict(val_X)
    print_step('Predict Test 2/2')
    pred_test_y = model.predict(test_X)
    return pred_val_y, pred_test_y

In [25]:
results = run_cv_model(train_, test_, target, runMLP, rmse)

[2018-06-26 22:26:12.572093] Started  fold 1/5
Train on 1202739 samples, validate on 300685 samples
Epoch 1/4
 - 23s - loss: 0.0447 - mean_squared_error: 0.0447 - val_loss: 0.0447 - val_mean_squared_error: 0.0447
Epoch 2/4
 - 22s - loss: 0.0442 - mean_squared_error: 0.0442 - val_loss: 0.0449 - val_mean_squared_error: 0.0449
Epoch 3/4
 - 22s - loss: 0.0441 - mean_squared_error: 0.0441 - val_loss: 0.0445 - val_mean_squared_error: 0.0445
Epoch 4/4
 - 23s - loss: 0.0440 - mean_squared_error: 0.0440 - val_loss: 0.0445 - val_mean_squared_error: 0.0445
[2018-06-26 22:27:43.245957] Loading best model
[2018-06-26 22:27:43.271763] Predict Val 1/2
[2018-06-26 22:28:02.184540] Predict Test 2/2
[2018-06-26 22:28:35.242447]  cv score 1 : 0.21084402971692776
[2018-06-26 22:28:35.251862] Started  fold 2/5
Train on 1202739 samples, validate on 300685 samples
Epoch 1/4
 - 23s - loss: 0.0447 - mean_squared_error: 0.0447 - val_loss: 0.0442 - val_mean_squared_error: 0.0442
Epoch 2/4
 - 22s - loss: 0.0443 -

In [26]:
print('~~~~~~~~~~')
print_step('Cache')
save_in_cache('MLP_blender', pd.DataFrame({'MLP_blender': results['train']}),
                           pd.DataFrame({'MLP_blender': results['test'].flatten()}))

~~~~~~~~~~
[2018-06-26 22:41:40.469998] Cache
[2018-06-26 22:41:43.209133] Saved cache/train_MLP_blender.csv and cache/test_MLP_blender.csv to cache!


In [21]:
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print_step('Prepping submission file')
submission = pd.read_csv(f'data/test.csv', usecols=['item_id'])
submission['deal_probability'] = results['test'].clip(0.0, 1.0)
submission.to_csv('submit/submit_MLP_blender.csv', index=False)
print_step('Done!')

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
[2018-06-19 22:50:04.882865] Prepping submission file
[2018-06-19 22:50:06.778460] Done!
